Imports

In [ ]:
import chess
import chess.engine
import random
import chess.svg
import time 
from IPython.display import display, HTML, clear_output

In [ ]:
path_to_stockfish = "/Applications/stockfish-10-mac/Mac/stockfish-10-popcnt"

Define start values

In [ ]:
positionsAnalyzedAlphaBeta = 0
positionsAnalyzedMinimax = 1
cutOffs = 0
currentEval = 0

Increment helper functions for Alpha-Beta and Minimax

In [ ]:
def incrementAlphaBeta():
    global positionsAnalyzedAlphaBeta
    positionsAnalyzedAlphaBeta = positionsAnalyzedAlphaBeta + 1
    
def incrementMinimax():
    global positionsAnalyzedMinimax
    positionsAnalyzedMinimax = positionsAnalyzedMinimax + 1

Helper function to reset Alpha-Beta and Minimax analysis

In [ ]:
def reset():
    global positionsAnalyzedAlphaBeta
    positionsAnalyzedAlphaBeta = 0
    positionsAnalyzedMinimax = 0

Maybe unnecessary chessboard creation

In [ ]:
board = chess.Board()  # can it be removed?

Get users next move

In [ ]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

Wrapper for the humans turn

In [ ]:
def human_player(board):
    display(board)
    uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print(f"Legal moves: {(', '.join(sorted(legal_uci_moves)))}")
        uci = get_move(f"{who(board.turn)}'s move [q to quit]>")
    return uci

Get move of random player

In [ ]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

AI 1 player

In [ ]:
def ai1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True)
    return moves[0].uci()

Stockfish

In [ ]:
def stockfish(board):
    engine = chess.engine.SimpleEngine.popen_uci(path_to_stockfish)
    result = engine.play(board, chess.engine.Limit(time=0.3))
    
    return result.move.uci()

Alpha-Beta for AI

In [ ]:
def aiAlphaBeta(board, color, depth):
    global currentEval
    allMoves = list(board.legal_moves)
    for move in allMoves:
        board.push(move)
        if board.is_checkmate():
            move.score = color * 1000
        elif board.is_stalemate():
            move.score = 0
        else:
            move.score = alphaBeta(board, -100000, +100000, -color, depth - 1)
        board.pop()
    sortedMoves = sorted(allMoves, key=lambda move: move.score)
    
    if color == -1:
        currentEval = sortedMoves[0].score
        return sortedMoves[0].uci()
    else:
        currentEval = sortedMoves[-1].score
        return sortedMoves[-1].uci()

Minimax for AI

In [ ]:
def aiMinimax(board, color, depth):
    allMoves = list(board.legal_moves)
    for move in allMoves:
        newboard = board.copy()
        newboard.push(move)
        if newboard.is_checkmate():
            move.score = color * 100
        elif newboard.is_stalemate():
            move.score = 0
        else:
            move.score = minimax(newboard, -color, depth - 1)
    if color == -1:
        return sorted(allMoves, key=lambda move: move.score)[0].uci()
    else:
        return sorted(allMoves, key=lambda move: move.score)[-1].uci()

Minimax

In [ ]:
def minimax(board, color, depth):
    incrementMinimax()
    
    if depth == 0:
        return staticAnalysis(board)
    else:
        allMoves = list(board.legal_moves)
        
        if color == 1:
            score = -color * 1000
            for move in allMoves:
                newboard = board.copy()
                newboard.push(move)
                
                if newboard.is_checkmate():
                    move.score = color * 100
                elif newboard.is_stalemate():
                    move.score = 0
                else:
                    move.score = minimax(newboard, -color, depth - 1)
                
                score = max(move.score, score)
        else:
            score = -color * 1000
            for move in allMoves:
                newboard = board.copy()
                newboard.push(move)
                
                if newboard.is_checkmate():
                    move.score = color * 100
                elif newboard.is_stalemate():
                    move.score = 0
                else:
                    move.score = minimax(newboard, -color, depth - 1)
                
                score = min(move.score,score)
        return score

Alpha-Beta

In [ ]:
def alphaBeta(board, alpha, beta, color, depth):
    incrementAlphaBeta()
    global cutOffs
    if depth == 0:
        return staticAnalysis(board)
    else:
        allMoves = list(board.legal_moves)
        allMoves = sortMoves(allMoves, board)
        if color == 1:
            score = -color * 1000
            for move in allMoves:
                board.push(move)
                if board.is_checkmate():
                    move.score = color * 1000
                elif board.is_stalemate():
                    move.score = 0
                else:
                    move.score = alphaBeta(board, alpha, beta, -color, depth - 1)
                board.pop()
                score = max(alpha, move.score)
                alpha = max(alpha, move.score)
                if alpha >= beta:
                    cutOffs += 1
                    break
        else:
            score = -color * 1000
            for move in allMoves:
                board.push(move)
                if board.is_checkmate():
                    move.score = color * 1000
                elif board.is_stalemate():
                    move.score = 0
                else:
                    move.score = alphaBeta(board, alpha, beta, -color, depth - 1)
                board.pop()
                score = min(score, move.score)
                beta = min(beta, move.score)
                if alpha >= beta:
                    cutOffs += 1
                    break
        return score

Static Analysis

In [ ]:
def staticAnalysis(board):
    score = 0
    for square in range(0, 64):
        piece = board.piece_at(square)
        if piece == None:
            score += 0
        elif piece.piece_type == 1:
            if piece.color == True:
                score += 1 + square / (8 * 10)
            else:
                score -= 1 + 0.8 - square / (8 * 10)
        elif piece.piece_type == 3:
            if piece.color == True:
                score += 3.5
            else:
                score -= 3.5
        elif piece.piece_type == 2:
            if piece.color == True:
                score += 3
            else:
                score -= 3
        elif piece.piece_type == 4:
            if piece.color == True:
                score += 5
            else:
                score -= 5
        elif piece.piece_type == 5:
            if piece.color == True:
                score += 9.5
            else:
                score -= 9.5
        elif piece.piece_type == 6:
            if piece.color == True:
                score += 100
                if square == 6:
                    score += 1
            else:
                score -= 100
                if square == 62:
                    score -= 1
    return score

Return a string representation of the player

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

Helper function to display the chess board

In [ ]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return f"<pre>{str(board)}</pre>"

Helper function to sort the moves

In [ ]:
def sortMoves(moves,board):
    tempCopyMoves = []
    for move in moves:
        if board.is_capture(move):
            tempCopyMoves.insert(0, move)
        else:
            tempCopyMoves.append(move)
    return tempCopyMoves

Function coordinating the game. It's the entry point for the game.

In [ ]:
def play_game(player1, player2, visual="svg", pause=0.3):
    use_svg = (visual == "svg")
    board = chess.Board()
    
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board, 1, 3)
                minimax(board, 1, 3)
            else:
                uci = player2(board, -1, 3)
                minimax(board, -1, 3)
                
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = f"<b> Move {len(board.move_stack)} {name}, Play '{uci}': </b><br/>{board_stop} </b><br/>All Positions: {positionsAnalyzedMinimax}, After pruning: {positionsAnalyzedAlphaBeta}, Positions Cut: {1 - positionsAnalyzedAlphaBeta / positionsAnalyzedMinimax} </b><br/>Current Eval {currentEval} </b><br/>CutOffs {cutOffs}"
            reset()
            
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted"
        return (None, msg, board)
    
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: fivefold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)

Start a game

In [ ]:
play_game(aiAlphaBeta, aiAlphaBeta)